In [1]:
import sys
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QPushButton, QLabel, QFileDialog, QStackedWidget, QProgressBar,
    QGridLayout, QGroupBox, QTabWidget
)
from PyQt5.QtGui import QPixmap, QIcon
from PyQt5.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

class HomePage(QWidget):
    def __init__(self):
        super().__init__()
        layout = QVBoxLayout()
        layout.setSpacing(10)
        layout.setContentsMargins(20, 20, 20, 20)

        logo_label = QLabel()
        script_dir = os.path.dirname(os.path.abspath(__file__))
        #logo_path = os.path.join(script_dir, "assets", "logo.png")
        logo_path = r"/Users/moon/Documents/Posture/Software/logo"
        if os.path.exists(logo_path):
            logo_pixmap = QPixmap(logo_path)
            logo_label.setPixmap(logo_pixmap.scaled(300, 300, Qt.KeepAspectRatio, Qt.SmoothTransformation))
        else:
            logo_label.setText("Logo not found")
        logo_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(logo_label)

        tab_widget = QTabWidget()
        tab_widget.addTab(self.create_recordings_tab(), "Recording")
        layout.addWidget(tab_widget)

        layout.addStretch()
        self.setLayout(layout)

    def create_recordings_tab(self):
        tab = QWidget()
        grid_layout = QGridLayout()

        recording_box = QGroupBox("RECORDINGS")
        recording_layout = QVBoxLayout()
        self.start_button = QPushButton("START")
        self.start_button.setIcon(QIcon("assets/play-icon.png"))
        self.start_button.clicked.connect(self.collect_data)
        self.stop_button = QPushButton("STOP")
        self.stop_button.setIcon(QIcon("assets/stop-icon.png"))
        self.stop_button.clicked.connect(self.stop_collect_data)
        recording_layout.addWidget(self.start_button)
        recording_layout.addWidget(self.stop_button)
        recording_box.setLayout(recording_layout)
        grid_layout.addWidget(recording_box, 0, 0)

        battery_box = QGroupBox("SENSOR BATTERY")
        battery_layout = QVBoxLayout()

        self.battery1_label = QLabel("S1 - Connected")
        self.battery1_icon = QLabel()
        self.battery1_icon.setPixmap(QPixmap("assets/battery-empty.png").scaled(40, 20, Qt.KeepAspectRatio))
        self.battery1_warning = QLabel("! Charge sensor")
        self.battery1_warning.setStyleSheet("color: red; font-weight: bold;")

        self.battery2_label = QLabel("S2 - Connected")
        self.battery2_icon = QLabel()
        self.battery2_icon.setPixmap(QPixmap("assets/battery-empty.png").scaled(40, 20, Qt.KeepAspectRatio))
        self.battery2_warning = QLabel("! Charge sensor")
        self.battery2_warning.setStyleSheet("color: red; font-weight: bold;")

        battery_layout.addWidget(self.battery1_label)
        battery_layout.addWidget(self.battery1_icon)
        battery_layout.addWidget(self.battery1_warning)
        battery_layout.addWidget(self.battery2_label)
        battery_layout.addWidget(self.battery2_icon)
        battery_layout.addWidget(self.battery2_warning)

        self.battery_update_btn = QPushButton("Simulate Battery Change")
        self.battery_update_btn.clicked.connect(self.update_battery_levels)
        battery_layout.addWidget(self.battery_update_btn)

        battery_box.setLayout(battery_layout)
        grid_layout.addWidget(battery_box, 0, 1)

        tab.setLayout(grid_layout)
        return tab

    def collect_data(self):
        print('Data Being Taken')

    def stop_collect_data(self):
        print('Data has been stopped')

    def update_battery_levels(self):
        def update_icon(level, icon_label, warning_label):
            if level < 20:
                icon_file = "battery-empty.png"
                warning_label.setText("! Charge sensor")
                warning_label.setStyleSheet("color: red; font-weight: bold;")
            else:
                if level < 40:
                    icon_file = "battery-20.png"
                elif level < 60:
                    icon_file = "battery-40.png"
                elif level < 80:
                    icon_file = "battery-60.png"
                elif level < 100:
                    icon_file = "battery-80.png"
                else:
                    icon_file = "battery-100.png"
                warning_label.setText("Sensor OK")
                warning_label.setStyleSheet("color: green;")

            icon_path = os.path.join("assets", icon_file)
            icon_label.setPixmap(QPixmap(icon_path).scaled(40, 20, Qt.KeepAspectRatio))

        new1 = random.randint(0, 100)
        new2 = random.randint(0, 100)
        update_icon(new1, self.battery1_icon, self.battery1_warning)
        update_icon(new2, self.battery2_icon, self.battery2_warning)

class PlotPage(QWidget):
    def __init__(self):
        super().__init__()

        self.label = QLabel("Click a button to load and plot data.")
        self.sensor_button = QPushButton("Load Sensor Data File")
        self.score_button = QPushButton("Load Score Data File")

        self.sensor_button.clicked.connect(self.load_sensor_file)
        self.score_button.clicked.connect(self.load_score_file)

        top_layout = QVBoxLayout()
        logo_label = QLabel()
        script_dir = os.path.dirname(os.path.abspath(__file__))
        logo_path = os.path.join(script_dir, "assets", "logo.png")
        if os.path.exists(logo_path):
            logo_pixmap = QPixmap(logo_path)
            logo_label.setPixmap(logo_pixmap.scaled(200, 200, Qt.KeepAspectRatio, Qt.SmoothTransformation))
        else:
            logo_label.setText("Logo not found")
        logo_label.setAlignment(Qt.AlignCenter)
        top_layout.addWidget(logo_label)

        self.figure1, self.ax1 = plt.subplots()
        self.canvas1 = FigureCanvas(self.figure1)

        self.figure2, self.ax2 = plt.subplots()
        self.canvas2 = FigureCanvas(self.figure2)

        layout = QVBoxLayout()
        layout.addLayout(top_layout)
        layout.addWidget(self.label)

        button_layout = QHBoxLayout()
        button_layout.addWidget(self.sensor_button)
        button_layout.addWidget(self.score_button)
        layout.addLayout(button_layout)

        layout.addWidget(QLabel("Sensor Data Plot"))
        layout.addWidget(self.canvas1)
        layout.addWidget(QLabel("Quarterly Score Trends"))
        layout.addWidget(self.canvas2)

        layout.addStretch()
        self.setLayout(layout)

    def load_sensor_file(self):
        path, _ = QFileDialog.getOpenFileName(self, "Open Sensor File", "", "CSV Files (*.csv);;Text Files (*.txt)")
        if path:
            try:
                df = pd.read_csv(path)
                print("Sensor file loaded:", df.columns.tolist())
                self.label.setText(f"Loaded sensor data: {os.path.basename(path)}")
                self.plot_sensor_data(df)
            except Exception as e:
                self.label.setText("Error loading sensor file")
                print("Sensor file load error:", e)

    def load_score_file(self):
        path, _ = QFileDialog.getOpenFileName(self, "Open Score File", "", "CSV Files (*.csv);;Text Files (*.txt)")
        if path:
            try:
                df = pd.read_csv(path)
                print("Score file loaded:", df.columns.tolist())
                self.label.setText(f"Loaded score data: {os.path.basename(path)}")
                self.plot_score_data(df)
            except Exception as e:
                self.label.setText("Error loading score file")
                print("Score file load error:", e)

    def plot_sensor_data(self, df):
        self.ax1.clear()
        if 'x' in df.columns and 'y' in df.columns:
            self.ax1.plot(df['x'], df['y'], label='x vs y')
            self.ax1.set_xlabel('X')
            self.ax1.set_ylabel('Y')
            self.ax1.set_title('Sensor Trajectory')
            self.ax1.legend()
        else:
            self.ax1.text(0.5, 0.5, "Missing 'x' or 'y' columns", ha='center', va='center')
        self.canvas1.draw()

    def plot_score_data(self, df):
        self.ax2.clear()
        if 'Date' in df.columns and 'Total Score' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
            df = df.dropna(subset=['Date'])
            df['Quarter'] = df['Date'].dt.to_period('Q')
            quarterly = df.groupby('Quarter')['Total Score'].mean()

            self.ax2.plot(quarterly.index.astype(str), quarterly.values, marker='o', linestyle='-')
            self.ax2.set_title("Average Total Score per Quarter")
            self.ax2.set_xlabel("Quarter")
            self.ax2.set_ylabel("Avg Score")
            self.ax2.grid(True)
        else:
            self.ax2.text(0.5, 0.5, "Missing 'Date' or 'Total Score' columns", ha='center', va='center')
        self.canvas2.draw()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("ALiOR")
        self.resize(800, 600)
        self.setStyleSheet("background-color: #ffffff;")

        self.sidebar = QWidget()
        sidebar_layout = QVBoxLayout()
        self.home_btn = QPushButton("Home")
        self.plot_btn = QPushButton("Analytics")

        self.home_btn.clicked.connect(lambda: self.pages.setCurrentIndex(0))
        self.plot_btn.clicked.connect(lambda: self.pages.setCurrentIndex(1))

        sidebar_layout.addWidget(self.home_btn)
        sidebar_layout.addWidget(self.plot_btn)
        sidebar_layout.addStretch()
        self.sidebar.setLayout(sidebar_layout)
        self.sidebar.setFixedWidth(150)
        self.sidebar.setStyleSheet("background-color: #f0f0f0;")

        self.pages = QStackedWidget()
        self.home_page = HomePage()
        self.plot_page = PlotPage()
        self.pages.addWidget(self.home_page)
        self.pages.addWidget(self.plot_page)

        main_widget = QWidget()
        main_layout = QHBoxLayout()
        main_layout.addWidget(self.sidebar)
        main_layout.addWidget(self.pages)
        main_widget.setLayout(main_layout)

        self.setCentralWidget(main_widget)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    app.setStyleSheet("""
        QWidget {
            font-family: 'Segoe UI', sans-serif;
            background-color: #ffffff;
            color: #222222;
        }
        QPushButton {
            background-color: #f0f0f0;
            border: 1px solid #cccccc;
            padding: 6px 12px;
            border-radius: 6px;
        }
        QPushButton:hover {
            background-color: #e6e6e6;
        }
        QLabel {
            font-size: 14px;
        }
        QGroupBox {
            font-weight: bold;
            border: 1px solid #cccccc;
            border-radius: 5px;
            margin-top: 10px;
        }
        QGroupBox::title {
            subcontrol-origin: margin;
            left: 10px;
            padding: 0 5px 0 5px;
        }
    """)
    window = MainWindow()
    window.setWindowTitle("ALIOR Interface")
    window.show()
    sys.exit(app.exec_())

NameError: name '__file__' is not defined